In [24]:
# 加载数据，把date这一列设置为索引，简单起见，只用收盘价进行分析
import numpy as np
import pandas as pd

# 导入数据
df = pd.read_csv('./20191019_biduStack.csv', index_col='date', usecols=[0, 1])
# df.head()

# 将索引转换为 Datetime 形式
df.index = pd.DatetimeIndex(df.index.str.strip("'"))

# 按照日期升序排序
df.sort_index(inplace=True)

df.describe()
df.head()

# 策略一：股价超出10日均线买入，跌破十日均线卖出
# 使用窗函数计算10日均线数据
ma10 = df.rolling(10).mean().dropna()

# 买点
ma10_model = df['close'] - ma10['close'] > 0
ma10_model


# 第一个值 False，第二个值是 True，在 True 时买入，需自定义移动窗口处理函数
# 因为卖时还需定义类似函数，所以这里把这两个函数放在一起
def get_deal_date(w, is_buy=True):
    if is_buy == True:
        return True if w[0] == False and w[1] == True else False
    else :
        return True if w[0] == True and w[1] == False else False

# raw=False 没有的话会有警告信息
# 如果删除 Na 值，会有缺失，所以这里用 0 填充，转换为 bool 值以便后面取值
se_buy = ma10_model.rolling(2).apply(get_deal_date, raw=False).fillna(0).astype('bool')

# apply 的 args 接受数组或者字典给自定义函数传参
se_sale = ma10_model.rolling(2).apply(get_deal_date, raw=False, args=[False]).fillna(0).astype('bool')

# 具体的买卖点
buy_info = df[se_buy.values]
sale_info = df[se_sale.values]
# print("具体买点是：", buy_info.head())
# print("\n")
# print("具体买点是：",sale_info.head())

# 买和卖的索引值不一样，不过数据都有 65 条，所以删除时间索引信息
no_index_buy_info = buy_info.reset_index(drop=True)
no_index_sale_info = sale_info.reset_index(drop=True)

# 每次交易的盈利情况
profit = no_index_sale_info - no_index_buy_info
# print(profit)

# 大体数据
profit.describe()

profit.sum()

# 假如有 1w 美元，最终盈利是多少
all_money = 10000
remain = all_money
for i in range(len(no_index_buy_info)):
    buy_count = remain / no_index_buy_info.iloc[i]
    remain = buy_count * no_index_sale_info.iloc[i]
    # all_money = sale_total - all_money
print(remain)


# # 如果加上每次交易金额的万分之三手续费
# all_money = 10000
# remain = all_money
# fee = 0.0003
# for i in range(len(no_index_buy_info)):
#     buy_count = remain / no_index_buy_info.iloc[i]
#     remain = buy_count * no_index_sale_info.iloc[i] * (1 - fee)
#     # all_money = sale_total - all_money
#     print(remain)


# 考虑一种边界情况，买了之后卖点到现在还没有出现（因为是先买的，所以不可能买点比卖点多）
# 这种情况我们把当前的股价加到卖点数据中（也可以把买点数据删除）
print(len(no_index_buy_info))
print(len(no_index_sale_info))
if (len(no_index_buy_info) > len(no_index_sale_info)):
    # no_index_buy_info.drop(no_index_buy_info.index[-1], inplace=True)
    no_index_sale_info.loc[len(no_index_sale_info)] = [df.iloc[-1, 0]]

close    9534.338029
dtype: float64
65
65
